In [1]:
import sys
from pathlib import Path

# Add project root to path
root = Path.cwd()
while not (root / "src").exists() and root != root.parent:
    root = root.parent
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

print(f"Project root: {root}")
print(f"src exists: {(root / 'src').is_dir()}")

# Imports
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from src.pipelines import HotelCancellationClassification
from src.utils import ModelMetadata

# Configuration
DATA_PATH = root / "data" / "processed" / "hotel_bookings_processed.csv"
TARGET_COL = "is_canceled"
RANDOM_STATE = 42

# Verify data exists
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Processed data not found: {DATA_PATH}")

print(f"\nData path: {DATA_PATH}")
print(f"File size: {DATA_PATH.stat().st_size / 1024 / 1024:.2f} MB")

Project root: /Users/otosamada/Documents/University Assignmnets/Programming/Project/DABN13---Project
src exists: True

Data path: /Users/otosamada/Documents/University Assignmnets/Programming/Project/DABN13---Project/data/processed/hotel_bookings_processed.csv
File size: 32.92 MB


In [2]:
df = pd.read_csv(DATA_PATH)

# One-hot encode categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
if TARGET_COL in categorical_cols:
    categorical_cols.remove(TARGET_COL)

if categorical_cols:
    print(f"One-hot encoding {len(categorical_cols)} categorical columns: {categorical_cols}")
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    
    # Save preprocessed data
    df.to_csv(DATA_PATH, index=False)
    print(f"Saved preprocessed data with {df.shape[1]} features")

print(f"Data shape: {df.shape}")
print(f"\nTarget distribution:\n{df[TARGET_COL].value_counts(normalize=True)}")
print(f"\nFeature types:\n{df.dtypes.value_counts()}")
df.head()

Data shape: (86544, 69)

Target distribution:
is_canceled
0    0.724025
1    0.275975
Name: proportion, dtype: float64

Feature types:
bool       46
int64      17
float64     6
Name: count, dtype: int64


,is_canceled,children,babies,is_repeated_guest,required_car_parking_spaces,total_of_special_requests,stays_in_weekend_nights,stays_in_week_nights,previous_cancellations,previous_bookings_not_canceled,...,arrival_date_month_November,arrival_date_month_October,arrival_date_month_September,adults_cat_2,adults_cat_3+,total_of_special_requests_cat_1,total_of_special_requests_cat_2+,stays_in_weekend_nights_cat_1,stays_in_weekend_nights_cat_2,stays_in_weekend_nights_cat_3+
0,0,0.0,0,0,0,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False
1,0,0.0,0,0,0,0,0,0,0,0,...,False,False,False,True,False,False,False,False,False,False
2,0,0.0,0,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
3,0,0.0,0,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
4,0,0.0,0,0,0,1,0,2,0,0,...,False,False,False,True,False,True,False,False,False,False


In [3]:
analysis = HotelCancellationClassification(
    data_path=DATA_PATH,
    target_col=TARGET_COL,
    features=None,  # Use all columns except target
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=True,
    experiment_name="hotel_cancellation_v1"
)

# Create train/val/test split (70% train, 15% val, 15% test)
analysis.prepare_split(val_size=0.15)

Features: 22 numeric, 0 categorical
Split: train=56253 val=12982 test=17309 | TrainPos=0.276 ValPos=0.276 TestPos=0.276


In [4]:
print("=" * 70)
print("TRAINING BASELINE MODELS")
print("=" * 70)

# Logistic Regression (multiple regularization strengths)
print("\n1. Logistic Regression")
for C in [0.1, 1.0, 10.0]:
    print(f"   Training with C={C}...")
    analysis.train_logistic_regression(C=C, scale=True, random_state=RANDOM_STATE)

# Decision Tree (multiple depths)
print("\n2. Decision Tree")
for depth in [4, 6, 8, None]:
    print(f"   Training with max_depth={depth}...")
    analysis.train_decision_tree(max_depth=depth, random_state=RANDOM_STATE)

print("\n" + "=" * 70)
print("BASELINE TRAINING COMPLETE")
print("=" * 70)

TRAINING BASELINE MODELS

1. Logistic Regression
   Training with C=0.1...
   Training with C=1.0...
   Training with C=10.0...

2. Decision Tree
   Training with max_depth=4...
   Training with max_depth=6...
   Training with max_depth=8...
   Training with max_depth=None...

BASELINE TRAINING COMPLETE


In [ ]:
print("=" * 70)
print("GRID SEARCH OPTIMIZATION")
print("=" * 70)

# Logistic Regression Grid Search
print("\n1. Logistic Regression GridSearch")
analysis.grid_search(
    estimator=LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    param_grid={
        "C": [0.01, 0.1, 1.0, 10.0, 100.0],
        "penalty": ["l2"],
        "solver": ["lbfgs", "saga"]
    },
    scoring="f1",
    cv=5,
    use_validation=True
)

# Decision Tree Grid Search
print("\n2. Decision Tree GridSearch")
analysis.grid_search(
    estimator=DecisionTreeClassifier(random_state=RANDOM_STATE),
    param_grid={
        "max_depth": [3, 4, 5, 6, 8, 10, None],
        "min_samples_split": [2, 5, 10, 20],
        "min_samples_leaf": [1, 2, 4]
    },
    scoring="f1",
    cv=5,
    use_validation=True
)

print("\n" + "=" * 70)
print("GRID SEARCH COMPLETE")
print("=" * 70)

GRID SEARCH OPTIMIZATION

1. Logistic Regression GridSearch
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/miniconda3/lib/python3.13/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: T

In [ ]:
print("\n" + "=" * 70)
print("EXPERIMENT SUMMARY")
print("=" * 70)

analysis.metadata.summary()

# Find best models by different metrics
print("\n" + "-" * 70)
print("BEST MODELS BY METRIC")
print("-" * 70)

for metric in ["test_f1", "test_roc_auc", "test_accuracy"]:
    best = analysis.metadata.best(metric=metric)
    if best:
        print(f"\nBest by {metric}:")
        print(f"  Algorithm: {best['algorithm']}")
        print(f"  Params: {best['hyperparameters']}")
        print(f"  Score: {best['results'].get(metric, 'N/A'):.4f}")

In [ ]:
// Seventh cell - Save results
# Save metadata to JSON
analysis.metadata.save()  # Saves to models/experiments/hotel_cancellation_v1.json

# Save best model by F1 score
analysis.metadata.save_best_model(metric="test_f1")

print("✓ Metadata saved to: models/experiments/hotel_cancellation_v1.json")
print("✓ Best model saved to: models/best_models/")

In [ ]:
// Eighth cell - Visualize best model performance
best = analysis.metadata.best(metric="test_f1")

if best:
    print(f"Evaluating best model: {best['algorithm']}")
    print(f"Hyperparameters: {best['hyperparameters']}")
    
    # Comprehensive evaluation report
    analysis.plot_evaluation_report(
        experiment=best,
        figsize=(16, 10),
        save_path="reports/figures/best_model_evaluation.png"
    )
else:
    print("No experiments found.")

In [ ]:
// Ninth cell - Feature importance (if decision tree is best)
best_tree = analysis.metadata.best(metric="test_f1", algorithm="DecisionTreeClassifier")

if best_tree:
    print("Decision Tree Feature Importance Analysis")
    analysis.display_feature_importance(
        experiment=best_tree,
        top_n=20,
        figsize=(12, 8)
    )
else:
    print("No decision tree experiments found.")

In [ ]:
// Tenth cell - Optional: XGBoost (if installed)
try:
    from xgboost import XGBClassifier
    
    print("=" * 70)
    print("XGBOOST GRID SEARCH")
    print("=" * 70)
    
    analysis.grid_search(
        estimator=XGBClassifier(
            objective="binary:logistic",
            eval_metric="logloss",
            tree_method="hist",
            random_state=RANDOM_STATE,
            verbosity=0
        ),
        param_grid={
            "n_estimators": [100, 200, 300],
            "max_depth": [3, 4, 6],
            "learning_rate": [0.01, 0.05, 0.1],
            "subsample": [0.8, 1.0],
            "colsample_bytree": [0.8, 1.0]
        },
        scoring="f1",
        cv=3,  # Fewer folds due to large param space
        use_validation=True
    )
    
    print("\n✓ XGBoost training complete")
    
    # Update best model check
    best_xgb = analysis.metadata.best(metric="test_f1")
    print(f"\nBest model after XGBoost: {best_xgb['algorithm']}")
    
except ImportError:
    print("XGBoost not installed. Skipping.")
    print("Install with: pip install xgboost")